In [207]:
import numpy as np
import pandas as pd

In [241]:
shot_chart = pd.read_csv('data/shot_chart_2022.csv')

In [242]:
names = [head for head in shot_chart.columns if head not in ['Game'] and not head.startswith("Unnamed")]
players = {}

In [243]:
shot_chart['Ian'].dropna().apply(lambda x: 'FT' in x).sum()

6

In [244]:
for name in names:
    player_shots = shot_chart[name].dropna()
    player_shots = player_shots[player_shots != "AND ONE"].apply(lambda x: x.strip().split(" "))
    player_shots = pd.DataFrame.from_dict(dict(zip(player_shots.index, player_shots.values))).T.reset_index(drop=True).rename(columns={0: 'region', 1: 'type'})
    player_shots['region'] = player_shots['region'].astype(int)
    # FIX THE FOLLOWING LINE TO HANDLE FREE THROWS!
    shot_type = player_shots['type']
    player_shots['fts'] = (shot_type == 'FT').astype(int) + (shot_type == '1FT').astype(int) + (shot_type == '2FT').astype(int)
    player_shots['made'] = (shot_type == 'M').astype(int)
    player_shots['att'] = (shot_type == 'A').astype(int) + player_shots['made']
    player_shots['att with fts'] = player_shots['att'] + player_shots['fts']
    stats = player_shots.groupby('region').sum().reset_index()
    stats['avg with ft'] = round(100 * (stats['made'] + stats['fts']) / stats['att with fts'], 1)
    stats['avg without ft'] = round(100 * stats['made'] / stats['att'], 1)
    players.update({name: stats})


In [253]:
pdam = pd.concat(players.values(), axis=0).groupby('region').sum().reset_index()
pdam['avg with ft'] = round(100 * (pdam['made'] + pdam['fts']) / pdam['att with fts'], 1)
pdam['avg without ft'] = round(100 * pdam['made'] / pdam['att'], 1)

,region,fts,made,att,att with fts,avg with ft,avg without ft
0,1,32,135,278,310,53.9,48.6
1,2,1,7,22,23,34.8,31.8
2,3,0,0,9,9,0.0,0.0
3,4,0,6,33,33,18.2,18.2
4,5,0,12,49,49,24.5,24.5
5,6,0,2,22,22,9.1,9.1
6,7,0,8,44,44,18.2,18.2
7,8,0,5,30,30,16.7,16.7
8,9,0,2,7,7,28.6,28.6
9,10,0,3,20,20,15.0,15.0


In [ ]:
writer = pd.ExcelWriter(mkdir('results/') + args.data_file.split('/')[-1].replace('.csv', '.xlsx'), engine='openpyxl')
for game, stat in stats.items():
    if game == 0:
        df_name = "season"
    else:
        df_name = "game {:d}".format(game)
    stat.round(decimals=1).to_excel(writer, sheet_name=df_name, index=False)
writer.save()

In [214]:
players

{'Enemy':     region  avg w/ ft  avg w/o ft
 0        1   0.728000    0.480000
 1        2   0.571429    0.285714
 2        3   0.000000    0.000000
 3        4   0.125000    0.125000
 4        5   0.266667    0.266667
 5        6   0.000000    0.000000
 6        7   0.052632    0.052632
 7        8   0.153846    0.153846
 8        9   0.500000    0.500000
 9       10   0.272727    0.272727
 10      11   0.250000    0.250000
 11      12   0.133333    0.133333
 12      13   0.000000    0.000000,
 'Drago':    region  avg w/ ft  avg w/o ft
 0       1   0.705882    0.470588
 1       2   0.666667    0.666667
 2       4   0.000000    0.000000
 3       5   0.500000    0.500000
 4       7   0.666667    0.666667
 5       8   0.000000    0.000000
 6      10   0.000000    0.000000
 7      12   2.000000    0.000000
 8      13   1.000000    1.000000,
 'Ansen':    region  avg w/ ft  avg w/o ft
 0       1   0.578947    0.473684
 1       2   0.250000    0.250000
 2       5   0.166667    0.166667
 3   

In [135]:
# count and %
pd.concat([player_shots.groupby('region').sum().rename(columns={'made': '# made'}), player_shots.groupby('region').mean().rename(columns={'made': '% made'})], axis=1).reset_index().sort_values(by=['region'])

,region,# made,% made
0,1,8,0.421053
1,10,0,0.000000
2,12,0,0.000000
3,13,1,1.000000
4,2,2,0.666667
5,4,0,0.000000
6,5,1,0.500000
7,7,2,0.666667
8,8,0,0.000000
